In [1]:
import pureml_evaluate

In [2]:
pureml_evaluate.__version__

'0.1.4.3'

In [3]:
from pureml_evaluate.evaluators.evaluator import Evaluator

In [4]:
import joblib
y_true = joblib.load("y_test.pkl").values
y_pred = joblib.load("y_pred.pkl")
sensitive_features = joblib.load("sensitive_features.pkl")

In [5]:
sensitive_features

9407       male
2311       male
22633    female
9255     female
8901     female
          ...  
26220    female
18170      male
25486    female
3113     female
27486      male
Name: SEX, Length: 10500, dtype: category
Categories (2, object): ['male', 'female']

In [6]:
print(sensitive_features.ndim, sensitive_features.shape)

1 (10500,)


In [7]:
import random
import numpy as np

values = ['Indian', 'African', 'American']

list_length = sensitive_features.shape[0]
full_list = values * (list_length // len(values))
full_list += values[:list_length % len(values)]
random.shuffle(full_list)

full_list = np.array(full_list)
print(full_list.shape)
# Print the resulting list
# print(full_list)

(10500,)


In [8]:
import pandas as pd
s_feat = pd.concat([sensitive_features.reset_index(drop=True), pd.DataFrame(full_list, columns=['Race'])], axis=1)

In [9]:
from pureml_evaluate.evaluators.evaluator import eval

metrics = {
            'classification':
                [
                    {'name': 'accuracy', 'threshold': 0.6},
                    {'name': 'precision', 'threshold': 0.6},
                ],
            'fairness':
                [
                    {'name': 'demographic_parity_difference', 'threshold': 0.1},
                    {'name': 'true_positive_parity', 'threshold': 0.1}
                    
                ]
        }

# metrics = None

values = eval(y_true=y_true, 
              y_pred=y_pred, 
              sensitive_features=s_feat, 
              evaluators=['classification', 'fairness'], metrics=metrics)
            #   evaluators=['classification'], metrics=metrics)
# values

Subset column:  SEX
Unique values in the column:  ['male', 'female']
Group:  male
Group count:  4170
Group:  female
Group count:  6330
Subset column:  Race
Unique values in the column:  ['African', 'American', 'Indian']
Group:  African
Group count:  3500
Group:  American
Group count:  3500
Group:  Indian
Group count:  3500
Sensitive Columns:  2 Total subsets:  5


In [10]:
import json


with open("values_pd_2.json", "w",) as out_file:
    json.dump(values, out_file)


In [11]:
with open("values_pd_2.json") as temp:
    v = json.load(temp)
    # v = json.loads(v)

print(type(v))
v

<class 'dict'>


{'__all__': {'column': [{'name': '__all__', 'value': '__all__'}],
  'metric': [{'name': 'accuracy',
    'category': 'performance',
    'value': 0.7971428571428572,
    'status': None,
    'columns_sensitive': None},
   {'name': 'precision',
    'category': 'performance',
    'value': 0.5270081164287713,
    'status': None,
    'columns_sensitive': None},
   {'name': 'demographic_parity_difference',
    'category': 'fairness',
    'value': 0.058073351745144164,
    'status': None,
    'columns_sensitive': [{'name': 'SEX', 'value': '__all__'}]},
   {'name': 'true_positive_parity',
    'category': 'fairness',
    'value': 0.11047349698537057,
    'status': None,
    'columns_sensitive': [{'name': 'SEX', 'value': '__all__'}]},
   {'name': 'demographic_parity_difference',
    'category': 'fairness',
    'value': 0.007142857142857173,
    'status': None,
    'columns_sensitive': [{'name': 'Race', 'value': '__all__'}]},
   {'name': 'true_positive_parity',
    'category': 'fairness',
    'valu